<a href="https://colab.research.google.com/github/harttu/nlpHarkka/blob/master/introNLPloppuprojektiRegression.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Intro to NLP coursework

##Getting data  from the server
* Lemmatized version performs slightly better


In [4]:
file = ""
!wget https://raw.githubusercontent.com/harttu/nlpHarkka/master/outputRegression.tsv
file = "outputRegression.tsv"


--2020-03-17 15:48:07--  https://raw.githubusercontent.com/harttu/nlpHarkka/master/outputRegression.tsv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 151.101.0.133, 151.101.64.133, 151.101.128.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|151.101.0.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 212264 (207K) [text/plain]
Saving to: ‘outputRegression.tsv.1’

outputRegression.ts 100%[===================>] 207.29K  --.-KB/s    in 0.04s   

2020-03-17 15:48:07 (4.59 MB/s) - ‘outputRegression.tsv.1’ saved [212264/212264]



## Building datamodel 
 * Including neutral label, makes model perform very poorly
 

In [48]:
import json
import random
import csv
import numpy as np

data = []
with open(file) as csv_file:
    csv_reader = csv.reader(csv_file, delimiter='\t')
    for row in csv_reader:
      data.append({"value":float(row[0]),"aspect":row[1],"before":row[2],"subject":row[3],"after":row[4], "text":row[2]+" "+row[3]+" "+row[4]})

random.shuffle(data)
print("class values:", data[0]["value"])
print("text:",data[0]["text"])

class values: 1.0
text:  XMAS_EVET10 olla nyt nousta lukuinen uudelleen#kuuntelu myötä minä henkilö#kohtaisesti yksi hyvä Aphex Twin biisi ikinä kun eka kerta kuunnella levy niin jäädä tuo kohtaan levy läpi käyminen jumiin kun pitää kuunnella joku 3-4 kerta uudestaan heti :D


In [49]:
# We need to gather the texts and labels into separate lists
texts=[one_example["text"] for one_example in data]
afters = [one_example["after"] for one_example in data]
subjects = [one_example["subject"] for one_example in data]
befores = [one_example["before"] for one_example in data]

values=[one_example["value"] for one_example in data]

print("This many texts",len(texts))
print("This many values",len(values))
print()
for value,text in list(zip(values,texts))[:20]:
    print(value,text[:50]+"...")



This many texts 955
This many values 955

1.0  XMAS_EVET10 olla nyt nousta lukuinen uudelleen#ku...
0.0 no huh ! ei tietää että Bustickets olla julkaista ...
0.0 about 30Gt kuulostaa aika vähältä kyllä Windowsill...
0.6666666666666666 olla muuten hieno katsella joskus puulin pelejä ES...
0.0 tämä ei pitää vähätellä edes huuliharpun kanssa kä...
0.3333333333333333 niin siksi ihmettelinkin miksi puhua juuri 21 mega...
1.0 Komea peli tuokin Itseäni kiinnostaa XM : O ja Cit...
0.0 Sattuuko kukaan tietään mikä saada käsiinsä hesen ...
0.0 Optima Service olla se toinen pulju ja se huoltaa ...
0.0 Naurattaa kyllä tämä jengin viha ubuntua kohtaan P...
0.0 Millä kielellä kannattaa ruveta vääntämään softa j...
1.0 Kahden eri firman osakkeeseen itse sijottaisin its...
1.0 2006 vuosi C6 olla ainakin ihan taivaallinen ajett...
1.0  Barca sentään voittaa jokin ei vain jokin vaan li...
-0.5 ilmeisesti läppärin käyttäminen auto olla uusi ! B...
0.0 Häh olla Cadillac tehdä etuvetoisia autoja 60 - lu...

In [0]:
#vectorizer=TfidfVectorizer(max_features=100000,binary=True,ngram_range=(1,1))
#feature_matrix=vectorizer.fit_transform(texts)

In [51]:
from sklearn.feature_extraction.text import CountVectorizer

vectorizer=CountVectorizer(max_features=100000,binary=True,ngram_range=(1,1))
feature_matrix=vectorizer.fit_transform(texts)
print("shape=",feature_matrix.shape)
print("what did we get? ->", feature_matrix.__class__)

shape= (955, 9902)
what did we get? -> <class 'scipy.sparse.csr.csr_matrix'>


In [52]:
print(vectorizer.get_feature_names()[:1000])

['000', '02', '04', '05', '07', '10', '100', '1000', '100e', '100km', '100tkm', '1080p', '10e', '10min', '11', '110', '1100', '112cdi', '112db', '12', '120', '1200e', '121', '123', '12c', '12volttia', '13', '130', '134a', '139e', '14', '140', '140099', '142mm', '144mm', '14e', '15', '150', '1500e', '150e', '150hv', '15tkm', '16', '1600', '16gb', '16x', '17', '175000', '18', '183', '185', '189e', '19', '190e', '1919', '1978', '1982', '1991', '1992', '1993', '1997', '19v', '1al', '1ma', '20', '200', '2000', '2000kg', '2002', '2004', '2006', '2009', '200ma', '2011', '2012', '2013', '2014', '2015', '2016', '2020', '204hv', '205', '206', '20v', '21', '211', '23', '24', '242', '248', '24h', '25', '250', '250hp', '256', '26', '2600k', '265', '267923185922621440', '26kansi', '27', '280', '280x', '284', '29', '290x', '2d', '2e', '2k', '2kki', '2paloa', '2vuotta', '30', '300', '3000', '300hp', '300tkm', '301', '308', '30gt', '30tkm', '31', '3138eo', '318', '32', '320', '325', '325i', '328', '33'

# Data split



In [0]:
from sklearn.model_selection import train_test_split

train_texts, dev_texts, train_values, dev_values=train_test_split(texts,values,test_size=0.2)
vectorizer=CountVectorizer(max_features=1000,binary=True,ngram_range=(1,1))
feature_matrix_train=vectorizer.fit_transform(train_texts)

feature_matrix_train=vectorizer.fit_transform(train_texts)
feature_matrix_dev=vectorizer.transform(dev_texts)

In [73]:
print(feature_matrix_train.shape)
print(feature_matrix_dev.shape)

(764, 1000)
(191, 1000)


# Classifier train


In [74]:
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import Ridge

reg = LinearRegression().fit(feature_matrix_train, train_values)
clf = Ridge(alpha=10)
clf.fit(feature_matrix_train, train_values)

Ridge(alpha=10, copy_X=True, fit_intercept=True, max_iter=None, normalize=False,
      random_state=None, solver='auto', tol=0.001)

In [75]:
print("LinearRegression")
print("DEV",reg.score(feature_matrix_dev, dev_values))
print("TRAIN",reg.score(feature_matrix_train, train_values))

print("Ridge")
print("DEV",clf.score(feature_matrix_dev, dev_values))
print("TRAIN",clf.score(feature_matrix_train, train_values))

LinearRegression
DEV -3.2807518622751513
TRAIN 0.999999999996582
Ridge
DEV -0.10579359253224241
TRAIN 0.5180492395354372


## Toy data for benchmarking

In [76]:
TOYtexts = texts
random.shuffle(TOYtexts)
TOYvalues = values
# classifier
TOYreg = LinearRegression().fit(feature_matrix_train, train_values)
# data split
TOYtrain_texts, TOYdev_texts, TOYtrain_values, TOYdev_values=train_test_split(TOYtexts,TOYvalues,test_size=0.2)
# vectorizer
TOYvectorizer=CountVectorizer(max_features=100000,binary=True,ngram_range=(1,1))
# training
TOYfeature_matrix_train=TOYvectorizer.fit_transform(TOYtrain_texts)
# apply model
TOYfeature_matrix_dev=TOYvectorizer.transform(TOYdev_texts)
# let the model try to do its best
TOYreg.fit(TOYfeature_matrix_train, TOYtrain_values)
# examine scores
print("TOYDEV",TOYreg.score(TOYfeature_matrix_dev, TOYdev_values))
print("TOYTRAIN",TOYreg.score(TOYfeature_matrix_train, TOYtrain_values))


TOYDEV -0.24900242778339512
TOYTRAIN 0.9999999999999594
